In [1]:
from skimage.io import imread
import numpy as np
import viso2
from pathlib import Path
from mayavi import mlab

In [2]:
# set the most relevant parameters
params = viso2.Stereo_parameters()
params.calib.f  = 721.5377
params.calib.cu = 609.5593
params.calib.cv = 172.854
params.base     = 0.537

# initialize visual odometry 
viso = viso2.VisualOdometryStereo(params)
#recon = viso2.Reconstruction()
#recon.setCalibration(params.calib.f, params.calib.cu, params.calib.cv)
matcher_params = viso2.Matcher_parameters()

matcher_params.f = 721.5377
matcher_params.cu = 609.5593
matcher_params.cv = 172.854
matcher_params.base = 0.537
matcher_params.nms_n = 13
matcher_params.nms_tau = 50
matcher_params.match_binsize = 50 
matcher_params.match_radius = 200
matcher_params.match_disp_tolerance = 2
matcher_params.outlier_flow_tolerance = 10
matcher_params.outlier_disp_tolerance = 10
matcher_params.multi_stage = 1
matcher_params.half_resolution = 0
matcher_params.refinement = 1

matcher = viso2.Matcher(matcher_params)
matcher.setIntrinsics(params.calib.f, params.calib.cu, params.calib.cv, params.base)

In [17]:
left_images = [f for f in Path("left").rglob("*.png")]
right_images = [f for f in Path("right").rglob("*.png")]

left_img = imread(left_images[0])
right_img = imread(right_images[0])

pose = viso2.Matrix_eye(4)

for i in range(1, 20):
    left_img = imread(left_images[i])
    right_img = imread(right_images[i])

    matcher.pushBack(left_img, right_img)

    left_img = imread("left/0000000081.png")
    right_img = imread("right/0000000081.png")

    matcher.pushBack(left_img, right_img)

    matcher.matchFeatures(2)
    matches = matcher.getMatches()
    matches_mat = np.empty([8, matches.size()])
    # print(matches.size())
    
    if viso.process_frame(left_img, right_img):
        motion = viso.getMotion()
        est_motion = viso2.Matrix_inv(motion)
        pose = pose * est_motion

        num_matches = viso.getNumberOfMatches()
        num_inliers = viso.getNumberOfInliers()
        print('Matches:', num_matches, "Inliers:", 100*num_inliers/num_matches, '%')
        print('Estimated motion:')
        print(est_motion)

        #matches = viso.getMatches()
        #recon.update(matches, motion, 0)
        
    else:
        print('.... failed!')
        
#points = recon.getPoints()
#print("Reconstructed", points.size(), "points...")

#if points.size() > 0:
#    pts = np.empty((points.size(),3))
#    for i,p in enumerate(points):
#        pts[i,:] = (p.x, p.y, p.z)

#    mlab.figure()
#    mlab.points3d(pts[:,0], pts[:,1], pts[:,2], colormap='copper')
#    mlab.show()

Matches: 356 Inliers: 89.04494382022472 %
Estimated motion:
   1.0000000   -0.0000724   -0.0000091    0.0001944 
   0.0000724    1.0000000    0.0002859   -0.0016383 
   0.0000091   -0.0002859    1.0000000    0.0005430 
   0.0000000    0.0000000    0.0000000    1.0000000 
Matches: 356 Inliers: 92.13483146067416 %
Estimated motion:
   1.0000000   -0.0000895   -0.0000114    0.0002298 
   0.0000895    1.0000000    0.0000545   -0.0001418 
   0.0000114   -0.0000545    1.0000000    0.0004024 
   0.0000000    0.0000000    0.0000000    1.0000000 
Matches: 356 Inliers: 91.85393258426966 %
Estimated motion:
   1.0000000   -0.0001164   -0.0000154    0.0002958 
   0.0001164    1.0000000    0.0001767   -0.0009776 
   0.0000154   -0.0001767    1.0000000    0.0006759 
   0.0000000    0.0000000    0.0000000    1.0000000 
Matches: 356 Inliers: 92.69662921348315 %
Estimated motion:
   1.0000000   -0.0001858   -0.0000186    0.0004430 
   0.0001858    1.0000000    0.0001795   -0.0008655 
   0.0000186   -0.